In [ ]:
import numpy as np
from itertools import product
from collections import defaultdict

n = 3

transformations = [
    lambda x: x,
    lambda x: np.fliplr(x),
    lambda x: np.flipud(x),
    lambda x: np.flipud(np.fliplr(x)),
    lambda x: np.transpose(x),
    lambda x: np.fliplr(np.transpose(x)),
    lambda x: np.flipud(np.transpose(x)),
    lambda x: np.flipud(np.fliplr(np.transpose(x))),
]

inverse_transformations = [
    lambda x: x,
    lambda x: np.fliplr(x),
    lambda x: np.flipud(x),
    lambda x: np.fliplr(np.flipud(x)),
    lambda x: np.transpose(x),
    lambda x: np.transpose(np.fliplr(x)),
    lambda x: np.transpose(np.flipud(x)),
    lambda x: np.transpose(np.fliplr(np.flipud(x))),
]


# transformations = [
#     lambda x: x,                         # Identity
#     lambda x: np.rot90(x, 1),            # Rotate 90°
#     lambda x: np.rot90(x, 2),            # Rotate 180°
#     lambda x: np.rot90(x, 3),            # Rotate 270°
#     lambda x: np.fliplr(x),              # Horizontal reflection
#     lambda x: np.flipud(x),              # Vertical reflection
#     lambda x: np.transpose(x),           # Diagonal reflection (TL-BR)
#     # lambda x: np.fliplr(np.transpose(x)) # Diagonal reflection (TR-BL)
# ]

# inverse_transformations = [
#     lambda x: x,                         # Identity
#     lambda x: np.rot90(x, 3),            # Rotate 90° inverse (rotate 270°)
#     lambda x: np.rot90(x, 2),            # Rotate 180° inverse
#     lambda x: np.rot90(x, 1),            # Rotate 270° inverse (rotate 90°)
#     lambda x: np.fliplr(x),              # Horizontal reflection inverse
#     lambda x: np.flipud(x),              # Vertical reflection inverse
#     lambda x: np.transpose(x),           # Diagonal reflection (TL-BR) inverse
#     # lambda x: np.transpose(np.fliplr(x)) # Diagonal reflection (TR-BL) inverse
# ]

original_actions = np.array(range(n * n)).reshape(n, n)
for i, transform in enumerate(transformations):
    assert (
        inverse_transformations[i](transform(original_actions)).flatten().tolist()
        == original_actions.flatten().tolist()
    )

transformed_actions = [transform(original_actions).flatten().tolist() for transform in transformations]
for i in range(len(transformed_actions)):
    for j in range(i + 1, len(transformed_actions)):
        assert transformed_actions[i] != transformed_actions[j]


def generate_win_conditions(width, height, win_length):
    win_conditions = []

    # Rows
    for row in range(height):
        for start_col in range(width - win_length + 1):  # Ensure win_length-length sequence fits
            win_conditions.append([start_col + row * width + offset for offset in range(win_length)])

    # Columns
    for col in range(width):
        for start_row in range(height - win_length + 1):  # Ensure win_length-length sequence fits
            win_conditions.append([col + (start_row + offset) * width for offset in range(win_length)])

    # Diagonals (top-left to bottom-right)
    for row in range(height - win_length + 1):
        for col in range(width - win_length + 1):  # Ensure win_length-length sequence fits
            win_conditions.append([(col + offset) + (row + offset) * width for offset in range(win_length)])

    # Diagonals (top-right to bottom-left)
    for row in range(height - win_length + 1):
        for col in range(win_length - 1, width):  # Ensure win_length-length sequence fits
            win_conditions.append([(col - offset) + (row + offset) * width for offset in range(win_length)])

    return win_conditions


def is_won(board, player, win_conditions):
    # Check if any win condition is satisfied
    for condition in win_conditions:
        if all(board[pos] == player for pos in condition):
            return True

    return False


win_conditions = generate_win_conditions(n, n, 3)


def generate_all_valid_boards():
    symbols = [" ", "X", "O"]
    all_boards = list(product(symbols, repeat=n * n))  # Generate all 3^(n*n) combinations
    assert (3 ** (n * n)) == len(all_boards)
    all_valid_boards = []

    for board in all_boards:
        x_count = board.count("X")
        o_count = board.count("O")

        # Valid boards must satisfy these conditions:
        if x_count == o_count or x_count == o_count + 1:
            # if not is_won(board, 'X', win_conditions) and not is_won(board, 'O', win_conditions):
            all_valid_boards.append(board)

    return all_valid_boards


def board_to_matrix(board):
    return np.array(board).reshape(n, n)


def matrix_to_board(matrix):
    return matrix.flatten().tolist()


def generate_symmetries(board):
    matrix = board_to_matrix(board)
    symmetries = [transform(matrix) for transform in transformations]
    return [matrix_to_board(sym) for sym in symmetries]


def get_canonical_representation(board):
    symmetries = generate_symmetries(board)
    min_symmetry = min(symmetries)
    return tuple(min_symmetry), symmetries.index(min_symmetry)


# Generate all empty positions on the board
def get_empty_positions(board):
    return [i for i, cell in enumerate(board) if cell == " "]


def display_board(board):
    print(f" {board[0]} | {board[1]} | {board[2]} ")
    print("---+---+---")
    print(f" {board[3]} | {board[4]} | {board[5]} ")
    print("---+---+---")
    print(f" {board[6]} | {board[7]} | {board[8]} ")
    print("\n")


def get_next_player(board):
    x_count = board.count("X")
    o_count = board.count("O")
    return "X" if x_count == o_count else "O"


def get_next_board(board, action):
    new_board = list(board)
    next_player = get_next_player(board)
    if next_player == "X":
        new_board[action] = "X"
    else:
        new_board[action] = "O"

    return tuple(new_board)


# Generate all valid Tic-Tac-Toe boards
all_valid_boards = generate_all_valid_boards()
state_action_pairs = 0
for valid_board in all_valid_boards:
    empty_positions = get_empty_positions(valid_board)
    state_action_pairs += len(empty_positions)

# Generate all canonical Tic-Tac-Toe boards
all_canonical_boards = set()
get_canonical_boards = {}
get_transform = {}
get_inverse_transform = {}
get_canonical_actions = {}
get_inverse_canonical_actions = {}
for valid_board in all_valid_boards:
    canonical_board, transform_idx = get_canonical_representation(valid_board)
    all_canonical_boards.add(canonical_board)
    get_canonical_boards[valid_board] = canonical_board
    get_transform[valid_board] = transformations[transform_idx]
    get_inverse_transform[valid_board] = inverse_transformations[transform_idx]
    get_inverse_canonical_actions[valid_board] = get_transform[valid_board](original_actions).flatten().tolist()
    get_canonical_actions[valid_board] = get_inverse_transform[valid_board](original_actions).flatten().tolist()

all_canonical_boards = sorted(list(all_canonical_boards))
all_canonical_actions = {}
canonical_state_action_pairs = 0
for canonical_board in all_canonical_boards:
    empty_positions = get_empty_positions(canonical_board)
    all_canonical_actions[canonical_board] = empty_positions
    canonical_state_action_pairs += len(empty_positions)


def get_canonical_board(board):
    return get_canonical_boards[tuple(board)]


def get_canonical_action(board, action):
    return get_canonical_actions[board][action]


def get_inverse_canonical_action(board, canonical_action):
    return get_inverse_canonical_actions[board][canonical_action]


def canonicalize(board, action):
    canonical_board = get_canonical_board(board)
    canonical_action = get_canonical_action(board, action)
    return canonical_board, canonical_action


canonical_board_to_next_canonical_board = {}
all_next_canonical_boards = set()
for canonical_board in all_canonical_boards:
    canonical_actions_to_next_canonical_board = {}
    for canonical_action in all_canonical_actions[canonical_board]:
        next_board = get_next_board(canonical_board, canonical_action)
        # if not is_won(next_board, 'X', win_conditions) and not is_won(next_board, 'O', win_conditions):
        next_canonical_board = get_canonical_board(next_board)
        all_next_canonical_boards.add(next_canonical_board)
        canonical_actions_to_next_canonical_board[canonical_action] = next_canonical_board

    canonical_board_to_next_canonical_board[canonical_board] = canonical_actions_to_next_canonical_board

qMatrix = defaultdict(lambda: -1)
for i, next_canonical_board in enumerate(all_next_canonical_boards):
    qMatrix[next_canonical_board] = i


def get(board=None, action=None):
    if board is None and action is None:
        return qMatrix
    if action is None and board is not None:
        actions = get_empty_positions(board)
        canonical_actions = [get_canonical_action(board, action) for action in actions]
        canonical_board = get_canonical_board(board)
        return [
            qMatrix[canonical_board_to_next_canonical_board[canonical_board][canonical_action]]
            for canonical_action in canonical_actions
        ]
    if action is not None and board is not None:
        return qMatrix[
            canonical_board_to_next_canonical_board[get_canonical_board(board)][get_canonical_action(board, action)]
        ]


def Qset(board, action, value):
    qMatrix[
        canonical_board_to_next_canonical_board[get_canonical_board(board)][get_canonical_action(board, action)]
    ] = value


def displayQ(valid_board):
    valid_actions = get_empty_positions(valid_board)
    canonical_board = get_canonical_board(valid_board)
    Qvalid_board = list(valid_board)
    for valid_action in valid_actions:
        Qvalid_board[valid_action] = qMatrix[
            canonical_board_to_next_canonical_board[canonical_board][get_canonical_action(valid_board, valid_action)]
        ]

    print("\n")
    print(np.array(Qvalid_board).reshape(n, n))


print(f"Number of boards:                               {(3**(n*n))}")
print(f"Number of state-action pairs:                   {(3**(n*n))*(n*n)}")
print(f"Number of valid boards:                         {len(all_valid_boards)}")
print(f"Number of valid state-action pairs:             {state_action_pairs}")
print(f"Number of canonical boards:                     {len(all_canonical_boards)}")
print(f"Number of canonical state-action pairs:         {canonical_state_action_pairs}")
print(f"Number of next canonical boards:                {len(all_next_canonical_boards)}")
print(f"Number of reduced canonical state-action pairs: {len(all_next_canonical_boards)}")

board1 = ("X", " ", " ", " ", "O", " ", " ", " ", " ")
board2 = (" ", " ", " ", " ", "O", "X", " ", " ", " ")
displayQ(board1)
displayQ(board2)
assert get(board1, 5) == get(board2, 0)
print(f"{get(board1, 5)} == {get(board2, 0)}")

Number of boards:                               19683
Number of state-action pairs:                   177147
Number of valid boards:                         6046
Number of valid state-action pairs:             19107
Number of canonical boards:                     1520
Number of canonical state-action pairs:         4808
Number of next canonical boards:                1073
Number of reduced canonical state-action pairs: 1073

In [ ]:
import numpy as np


def permutation_matrix(permutation):
    n = len(permutation)
    return np.eye(n, dtype=int)[permutation]


def is_group(matrix_set):
    identity = np.eye(matrix_set[0].shape[0])  # Identity matrix of appropriate size

    # 1. Check Closure
    for A in matrix_set:
        for B in matrix_set:
            if not any(np.allclose(A @ B, C) for C in matrix_set):
                return False, "Closure failed"

    # 2. Associativity (already guaranteed for matrices)

    # 3. Check Identity
    if not any(np.allclose(identity, A) for A in matrix_set):
        return False, "Identity missing"

    # 4. Check Inverses
    for A in matrix_set:
        try:
            A_inv = np.linalg.inv(A)
        except np.linalg.LinAlgError:  # Matrix is not invertible
            return False, f"Matrix {A} has no inverse"
        if not any(np.allclose(A_inv, B) for B in matrix_set):
            return False, f"Inverse of {A} is not in the set"

    return True, "The set forms a group"


def construct_group(matrices):
    group = set(tuple(mat.flatten()) for mat in matrices)  # Use set for uniqueness

    # Identity matrix of appropriate size
    size = matrices[0].shape[0]
    identity = np.eye(size)
    group.add(tuple(identity.flatten()))

    # Add inverses and closure under multiplication
    added = True
    while added:
        added = False
        current_group = list(group)

        # Closure: Multiply all pairs
        for mat1 in current_group:
            for mat2 in current_group:
                product = np.dot(np.array(mat1).reshape(size, size), np.array(mat2).reshape(size, size))
                if tuple(product.flatten()) not in group:
                    group.add(tuple(product.flatten()))
                    added = True

        # Add inverses
        for mat in current_group:
            mat_np = np.array(mat).reshape(size, size)
            try:
                inverse = np.linalg.inv(mat_np)
            except np.linalg.LinAlgError:
                continue  # Skip non-invertible matrices
            if tuple(inverse.flatten()) not in group:
                group.add(tuple(inverse.flatten()))
                added = True

    # Convert back to numpy array format
    group = [np.array(g).reshape(size, size) for g in group]
    return group


def rank_factorization(A, r):
    """
    Perform a rank-r factorization of matrix A using SVD.

    Parameters:
        A (numpy.ndarray): The input matrix (m x n).
        r (int): Desired rank for the factorization.

    Returns:
        U (numpy.ndarray): Matrix of dimensions (m x r).
        V (numpy.ndarray): Matrix of dimensions (r x n).
    """
    # Perform SVD
    U, S, VT = np.linalg.svd(A, full_matrices=False)

    # Truncate to rank r
    U_r = U[:, :r]
    S_r = np.diag(S[:r])  # Convert top r singular values to a diagonal matrix
    VT_r = VT[:r, :]

    # Construct the factorization
    U_factor = U_r @ S_r  # m x r matrix
    V_factor = VT_r  # r x n matrix

    return U_factor, V_factor

In [ ]:
transformationsud = [
    lambda x: x,  # Identity
    lambda x: np.flipud(x),  # Horizontal reflection
]

transformationslr = [
    lambda x: x,  # Identity
    lambda x: np.fliplr(x),  # Horizontal reflection
]

transformationsrot = [
    lambda x: x,  # Identity
    lambda x: np.rot90(x, 1),  # Rotate 90°
    lambda x: np.rot90(x, 2),  # Rotate 180°
    lambda x: np.rot90(x, 3),  # Rotate 270°
]

transformationslrud = [
    lambda x: x,  # Identity
    lambda x: np.fliplr(x),  # Horizontal reflection
    lambda x: np.flipud(x),  # Vertical reflection
    lambda x: np.flipud(np.fliplr(x)),  # Vertical reflection
]

transformations = [
    lambda x: x,  # Identity
    lambda x: np.fliplr(x),  # Horizontal reflection
    lambda x: np.flipud(x),  # Vertical reflection
    lambda x: np.flipud(np.fliplr(x)),  # Vertical reflection
    lambda x: np.transpose(x),  # Diagonal reflection (TL-BR)
    lambda x: np.fliplr(np.transpose(x)),  # Horizontal reflection
    lambda x: np.flipud(np.transpose(x)),  # Vertical reflection
    lambda x: np.flipud(np.fliplr(np.transpose(x))),  # Vertical reflection
]

N = 3
indices = np.array(list(range(N * N))).reshape(N, N)
permutations = [transform(indices).flatten().tolist() for transform in transformations]
Ps = np.array([permutation_matrix(permutation) for permutation in permutations])

result, message = is_group(Ps)
print(message)

group = construct_group(Ps)
print(len(group))

In [ ]:
PP = np.sum(group, axis=0)
print(PP)
print(np.linalg.matrix_rank(PP))

U, V = rank_factorization(PP, np.linalg.matrix_rank(PP))
threshold = 10e-10
V[np.abs(V) < threshold] = 0
U[np.abs(U) < threshold] = 0
U = U
V = V
print(U)
print(V)

In [ ]:
from sympy import symbols, Matrix

num_symbols = N**2  # Number of symbols you want to create
matrix_names = [f"w{i}{j}" for i in range(1, num_symbols + 1) for j in range(1, num_symbols + 1)]
matrix_symbols = symbols(matrix_names)
cell_names = [f"s{i}" for i in range(1, num_symbols + 1)]
cell_symbols = symbols(cell_names)
cell_vector = Matrix(cell_symbols)
W = Matrix([[matrix_symbols[j + (num_symbols) * i] for i in range(num_symbols)] for j in range(num_symbols)]).T
PP0 = Matrix(PP)
VV = Matrix(V)
display(PP0 * cell_vector)
display(VV * cell_vector)

In [ ]:
transformed_cell_vector = PP0 * cell_vector
np.array(transformed_cell_vector).reshape(3, 3)

In [ ]:
((PP // 2) @ [1, 0, 0, 0, 1, 0, 0, -1, -1]).reshape(3, 3)

In [ ]:
state_symbols = symbols(["a_+", "a_0", "a_-"])
all_states = list(product(state_symbols, repeat=N * N))
s0 = Matrix(all_states[1000])
V_int = (2 * V).astype(int)
VV = Matrix(V_int)
display(VV * s0)

In [ ]:
all_valid_states = []
for state in all_states:
    x_count = state.count(state_symbols[0])
    o_count = state.count(state_symbols[-1])
    if x_count == o_count or x_count == o_count + 1:
        all_valid_states.append(state)

all_transformed_states = []
for state in all_states:
    all_transformed_states.append(tuple(VV @ Matrix(state)))

print(len(set(all_transformed_states)))

set(all_transformed_states)

In [ ]:
symbols = [1, 2, 3]
all_states = np.array(list(product(symbols, repeat=N * N)))
print(all_states.shape)
all_valid_states = []
for state in all_states:
    x_count = list(state).count(2)
    o_count = list(state).count(3)

    # Valid boards must satisfy these conditions:
    if x_count == o_count or x_count == o_count + 1:
        # if not is_won(board, 'X', win_conditions) and not is_won(board, 'O', win_conditions):
        all_valid_states.append(state)

    # if sum(state) == 0 or sum(state) == 1:
    #     all_valid_states.append(state)

all_valid_states = np.array(all_valid_states)
print(all_valid_states.shape)
all_transformed_states = set()
for state in all_valid_states:
    all_transformed_states.add(tuple(V @ state))

all_transformed_states = np.array(list(all_transformed_states))
print(all_transformed_states.shape)

In [ ]:
qMatrix = defaultdict(lambda: -1)
board = (" ", " ", " ", " ", " ", " ", " ", " ", " ")
Qset(board, 0, 0.2)
displayQ(board)

In [ ]:
valid_board = all_valid_boards[5]
actions = get_empty_positions(valid_board)
canonical_actions = [get_canonical_action(valid_board, action) for action in actions]
print(f"Board:             {valid_board}")
print(f"Canonical board:   {get_canonical_board(valid_board)}")
print(f"Actions:           {actions}")
print(f"Canonical actions: {sorted(canonical_actions)}")
print(f"Canonical actions: {sorted(all_canonical_actions[get_canonical_board(valid_board)])}")

tot = 0
for i, valid_board in enumerate(all_valid_boards):
    actions = get_empty_positions(valid_board)
    canonical_board = get_canonical_board(valid_board)
    canonical_actions1 = sorted(all_canonical_actions[canonical_board])
    canonical_actions2 = sorted([get_canonical_action(valid_board, action) for action in actions])
    if not canonical_actions2 == canonical_actions1:
        tot += 1

print(f"Number of wrong canonical actions:      {tot}/{len(all_valid_boards)}")

In [ ]:
for i in range(10):
    if i == 10 - 1:
        print(f"final i: {i}")
    else:
        print(f"{i}, {i + 1}")

for i in reversed(range(10)):
    if i == 10 - 1:
        print(f"final i: {i}")
    else:
        print(f"{i}, {i + 1}")

In [ ]:
import collections

# initializing deque
de = collections.deque([1, 2, 3], maxlen=3)
print("deque: ", de)

# using append() to insert element at right end
# inserts 4 at the end of deque
de.append(4)

# printing modified deque
print("\nThe deque after appending at right is : ")
print(de)

# using appendleft() to insert element at left end
# inserts 6 at the beginning of deque
de.appendleft(6)

# printing modified deque
print("\nThe deque after appending at left is : ")
print(de)

In [ ]:
# Python3 program to demonstrate
# the use of sample() function .

# import random
import random


# Prints list of random items of
# length 3 from the given list.
list1 = [1, 2, 3, 4, 5, 6]
print("With list:", random.sample(list1, 3))
print("With list:", random.sample(list1, 2))

# Prints list of random items of
# length 4 from the given string.
string = "GeeksforGeeks"
print("With string:", random.sample(string, 4))

# Prints list of random items of
# length 4 from the given tuple.
tuple1 = ("ankit", "geeks", "computer", "science", "portal", "scientist", "btech")
print("With tuple:", random.sample(tuple1, 4))

In [ ]:
class LazyComputeDict(dict):
    def __init__(self, compute_func, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.compute_func = compute_func

    def __getitem__(self, key):
        if key not in self:
            # Compute and store the value if it doesn't exist
            self[key] = self.compute_func(key)
        return super().__getitem__(key)


def compute_value(key):
    # Replace this with the actual computation logic
    return key**2


# Usage
my_dict = LazyComputeDict(lambda key: key**2)

# Accessing a key
key = 50
value = my_dict[key]  # Computes and stores the value on the fly
print(value)


def create_level2_lazy_dict(compute_func):
    def level1_compute(outer_key):
        # Create another LazyComputeDict for the second level
        return LazyComputeDict(lambda inner_key: compute_func(outer_key, inner_key))

    # Return the top-level LazyComputeDict
    return LazyComputeDict(level1_compute)


# Create the level-2 LazyComputeDict
nested_dict = create_level2_lazy_dict(compute_value)

# Accessing values dynamically
print(nested_dict["group1"]["item1"])  # Triggers computation for ('group1', 'item1')
print(nested_dict["group2"]["item2"])  # Triggers computation for ('group2', 'item2')

# Accessing the entire structure
print(nested_dict)

In [ ]:
state_to_board_translation = {"X": 1, "O": -1, " ": 0}
board_to_state_translation = {}
for key, value in state_to_board_translation.items():
    print(f"{key} -> {value}")
    board_to_state_translation[value] = key

board_to_state_translation

In [ ]:
import random
from collections import deque

replay_buffer = deque(maxlen=100)
batch_size = 16
for i in range(100):
    replay_buffer.append(i)

print(replay_buffer)
print(random.sample(replay_buffer, batch_size))

In [ ]:
import matplotlib.pyplot as plt

# Sample data for demonstration (Replace these with actual `meanlossX` and `meanavg_action_valueX` values)
import numpy as np

meanlossX = np.random.rand(100)  # Replace with actual data
meanavg_action_valueX = np.random.rand(100)  # Replace with actual data

# Create a figure with two subplots next to each other
fig, axs = plt.subplots(1, 2, figsize=(12, 5))  # 1 row, 2 columns

# Plot the first graph: Mean Loss
axs[0].plot(meanlossX, label="Mean Loss")
axs[0].set_title("Mean Loss Over Time")
axs[0].set_xlabel("Time")
axs[0].set_ylabel("Loss")
axs[0].grid(True)
axs[0].legend()

# Plot the second graph: Mean Average Action Value
axs[1].plot(meanavg_action_valueX, label="Mean Avg Action Value", color="orange")
axs[1].set_title("Mean Avg Action Value Over Time")
axs[1].set_xlabel("Time")
axs[1].set_ylabel("Action Value")
axs[1].grid(True)
axs[1].legend()

# Adjust layout for better spacing
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
def is_nested(d):
    """Check if a dictionary is nested."""
    return any(isinstance(value, dict) for value in d.values())


def extract_values(d):
    """Extract all values from a potentially nested dictionary."""
    values = []
    for key, value in d.items():
        if isinstance(value, dict):  # If the value is a dictionary, recurse
            values.extend(extract_values(value))
        else:
            values.append(value)
    return values


# Example usage
data = {"a": 1, "b": {"c": 2, "d": {"e": 3, "f": 4}}, "g": 5}

# Check if the dictionary is nested
print("Is nested:", is_nested(data))

# Extract all values
all_values = extract_values(data)
print("All values:", all_values)

In [ ]:
import torch

cell_vector = torch.tensor([[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0]])
action = torch.tensor([8])
cell_vector.gather(1, action.unsqueeze(1))

In [ ]:
import numpy as np
import torch

states = (np.array([[0, 0, 0, 0, -1, 1, -1, 1, 0]]), np.array([[0, 0, 0, 0, -1, 1, 0, 0, 0]]))
actions = (5, 5)
states = torch.FloatTensor(states)
actions = torch.LongTensor(actions).unsqueeze(1)
print(states)
print(actions)
states.gather(2, actions.unsqueeze(2)).squeeze(2)
next_q_values = states.max(2, keepdim=True)[0].squeeze(2)
print(next_q_values)

In [ ]:
states = (np.array([[0, 0, 0, 0, -1, 1, -1, 1, 0]]), np.array([[0, 0, 0, 0, -1, 1, 0, 0, 0]]))
print(states)
states = np.array((np.array([[0, 0, 0, 0, -1, 1, -1, 1, 0]]), np.array([[0, 0, 0, 0, -1, 1, 0, 0, 0]])))
print(states)

In [ ]:
["X"] * 9

In [ ]:
# import dill

# with open('SymmetricQ_optimalX.pkl', 'wb') as f:
#     dill.dump(Q1.get(), f)

# with open('SymmetricQ_optimalO.pkl', 'wb') as f:
#     dill.dump(Q2.get(), f)

# with open('TotallySymmetricQ_optimalX.pkl', 'wb') as f:
#     dill.dump(Q1.get(), f)

# with open('TotallySymmetricQ_optimalO.pkl', 'wb') as f:
#     dill.dump(Q2.get(), f)

In [ ]:
import torch

tensor = torch.tensor([1, 2, 3])  # Shape: (3,)
unsqueezed_tensor = tensor.unsqueeze(0)  # Adds a new dimension at index 0
print(unsqueezed_tensor)  # Shape: (1, 3)

unsqueezed_tensor = tensor.unsqueeze(1)  # Adds a new dimension at index 1
print(unsqueezed_tensor)  # Shape: (3, 1)

In [ ]:
import torch

tensor = torch.tensor([[[1], [2], [3]]])  # Shape: (1, 3, 1)
squeezed_tensor = tensor.squeeze()  # Removes all size-1 dimensions
print(squeezed_tensor)  # Shape: (3,)

# Squeezing a specific dimension
squeezed_tensor = tensor.squeeze(0)  # Removes the size-1 dimension at index 0
print(squeezed_tensor)  # Shape: (3, 1)

In [ ]:
torch.backends.mps.is_built()
device = torch.device("mps")
tensor = torch.ones((3, 3), device=device)
print("Tensor on MPS device:", tensor)